# Création de la base Représentants et Personnes morales 

- Importation des données TC et TITMC 

- Codification des adresses 

- Ajout et traitement du code commune

- Retraiter et simplifier les qualités 

- joindre avec les données de décès

In [1]:
# fonction permettant de traiter les données rep et pm et d'ajouter le code commune insee
import os
import s3fs
import pandas as pd
import functions as fc 

In [ ]:
# Lecture de la base inpi
df_rep_pm = fc.import_inpi("data_rep_pm_2017.csv")

In [ ]:
# nettoyage de diverses variables
df_rep_pm = fc.clean_rep_pm_inpi(df_rep_pm)

In [ ]:
# ajout du code commune
df_rep_pm = fc.add_code_com(df_rep_pm)

In [ ]:
# export de la table
fc.export2minio("df_rep_pm_2017_commune.csv", df_rep_pm)

In [2]:
df_rep_pm = fc.import_inpi( "df_rep_pm_2017_commune.csv", cols=None)

/home/coder/work/data_inpi/code/functions.py:192: DtypeWarning: Columns (14,16,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8",


In [3]:
df_rep_pm.head()

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,pays,id_representant,top_civilite,nom_patronymique2,top_particule,qualite2,denomination2,commune,code_departement,code_commune_insee
0,101,15850944,AGIR TECHNOLOGIES,Société par actions simplifiée à associé unique,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,agir technologies,NaN,na,NaN
1,101,15851793,DORAS,Société anonyme à directoire et conseil de sur...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,doras,NaN,na,NaN
2,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Président,VATIN,NaN,Gerard,P.Physique,1947-10-07,...,FRANCE,1.0,False,VATIN,False,président,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,...,FRANCE,3.0,False,VATIN,False,directeur général,societe immobiliere financiere d etudes et de ...,paris,75,NaN
4,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,JORDENS,VATIN,Regina,P.Physique,1948-08-07,...,FRANCE,4.0,False,JORDENS,False,directeur général,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0


In [4]:
# on filtre on ne va garder que les lignes avec le nom patronymique, le nom usage, les prénoms, ou la date de naissance
df_rep_pm = df_rep_pm.loc[(df_rep_pm["nom_patronymique"].isna() == False) | (df_rep_pm["prenoms"].isna() == False) | \
                            (df_rep_pm["date_naissance"].isna() == False), :]

In [5]:
df_rep_pm.shape # 8 671 244

(8671244, 27)

In [6]:
test = df_rep_pm.loc[df_rep_pm["code_commune_insee"].isna() == False, :]

In [7]:
test.shape # 4 693 722 avec code commune

(4693722, 27)

In [8]:
test = df_rep_pm.loc[df_rep_pm["code_commune"].isna() == False, :]

In [9]:
test.shape  # 7 740 164

(7740164, 27)

In [11]:
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'top_civilite', 'nom_patronymique2', 'top_particule', 'qualite2',
       'denomination2', 'commune', 'code_departement', 'code_commune_insee',
       'code_commune3'],
      dtype='object')

In [10]:
# correction pour paris, marseille et lyon
df_rep_pm["code_commune3"] = df_rep_pm["code_commune"]

In [14]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), :].head()

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,id_representant,top_civilite,nom_patronymique2,top_particule,qualite2,denomination2,commune,code_departement,code_commune_insee,code_commune3
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,...,3.0,False,VATIN,False,directeur général,societe immobiliere financiere d etudes et de ...,paris,75,NaN,NaN
486,101,304455298,ARNO AUTOMOBILES REPARATIONS NEUVES ET OCCASIONS,Société par actions simplifiée,Président du conseil de surveillance,BERNARD,NaN,Isabelle Jeanne,P.Physique,1958-10-06,...,3.0,False,BERNARD,False,président du conseil de surveillance,arno automobiles reparations neuves et occasions,paris,75,NaN,NaN
722,101,308491604,CONVERT SA,Société anonyme,Administrateur,CONVERT,NaN,Jean-Michel,P.Physique,1960-02-19,...,3.0,False,CONVERT,False,administrateur,convert sa,paris,75,NaN,NaN
910,101,310792023,ArcelorMittal Wire France,Société anonyme,Administrateur,PHILIPPE,NaN,Anne Suzanne Marie,P.Physique,1954-12-31,...,70.0,False,PHILIPPE,False,administrateur,arcelormittal wire france,paris,75,NaN,NaN
1216,101,313384133,SOCIETE CIVILE IMMOBILIERE DE LA FRANCHISE,Société civile immobilière,Associé indéfiniment responsable,VILLIERS,CHAVANE,Béatrice Marie,P.Physique,1964-03-15,...,2.0,False,VILLIERS,False,associé indéfiniment responsable,societe civile immobiliere de la franchise,paris,75,NaN,NaN


In [ ]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"] = "751" + \
    df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), "code_postal"].str.slice(start=4, stop=7)

In [ ]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "paris") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape  # 8 551

In [ ]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape # 945

In [ ]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"] = "6938" + \
    df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_postal"].str.slice(start=5, stop=7)

In [ ]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "lyon") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape # 945

In [ ]:
df_rep_pm.loc[(df_rep_pm["code_commune3"].isna()) & (df_rep_pm["commune"] == "marseille") & (df_rep_pm["code_postal"].isna() == False), "code_commune3"].shape # 2827